In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm

from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

Read **STEFAN HEIGLS** data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
f = np.load('../data/turbulentbox.npy')
data = f.copy()
del f

Read **THOMAS PFEILS** data. Data is currently (until 02.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/ebdbc12c85438bd224a5015206840893).

In [ ]:
with np.load('../data/pluto_data.npz') as f:
    data = f['rho']

# Visualization

## My own renderer

In [ ]:
import volrender

In [ ]:
vmax = data.max()
lognorm = LogNorm(vmin=vmax * 1e-4, vmax=vmax, clip=True)
datacube = lognorm(data.ravel()).reshape(data.shape).data

In [ ]:
ren = volrender.Renderer(np.moveaxis(datacube, 0, 1))
colors = ren.transferfunction.colors.copy()

ren.transferfunction.x0            = [0.1, 0.4, 0.9]
ren.transferfunction.sigma         = [0.05, 0.05, 0.1]
ren.transferfunction.colors        = colors
ren.transferfunction.colors[:, -1] = [0.1, 0.2, 0.1]
ren.transferfunction.A             = [0.2, 0.1, 0.3]

In [ ]:
ren.render(0, 50)

### Plots

In [ ]:
f, ax = ren.plot(norm=lognorm, diagnostics=True)

Same disk different transfer function

In [ ]:
ren.transferfunction.x0     = np.array([0.2, 0.88])
ren.transferfunction.sigma  = np.array([0.05, 0.02])
ren.transferfunction.colors = np.array([
    [1.  , .5  , 0.  , 0.1],
    [.25  , .25  , .75  , 0.1]])
ren.transferfunction.A      = np.array([1, 2])
ren.render(update=False)
ren.plot(diagnostics=False)

### Make a movie

due to the size, here we can just use 1 CPU or we get memory issues

In [ ]:
n_ang = 90
theta = np.linspace(45, 45, n_ang)
phi = np.linspace(0, 180, n_ang)
fname = 'movie_pluto.mp4'
volrender.render_helper.render_movie(ren.data, theta, phi, ncpu=1, tf=ren.transferfunction, fname=fname)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls>
  <source src="{fname}" type="video/mp4">
</video>
""")

## Pyvista

In [ ]:
import pyvista

In [ ]:
grid = pyvista.UniformGrid()
grid.dimensions = np.array(data.shape) + 1
grid.spacing = (1, 1, 1)
grid.cell_data["density"] = np.log10(data.flatten(order="F"))

vmax = grid['density'].max()
grid['opac'] = Normalize(vmin=vmax - 1.5, vmax=vmax)(grid['density'])

In [ ]:
pyvista.close_all()

In [ ]:
p = pyvista.Plotter()
p.set_background('white')
p.add_volume(grid, scalars='opac', cmap="viridis", clim=[0, 1], opacity='linear', shade=False, opacity_unit_distance=.010)
p.show()

## YT

In [ ]:
import yt

In [ ]:
plt.style.use({'figure.dpi':150})

In [ ]:
with np.load('pluto_data.npz') as f:
    data = f['rho']

In [ ]:
data_dict = dict(density = (data, "cm**-3"))
bbox = np.array([[-1, 1], [-1, 1], [-1, 1]])
ds = yt.load_uniform_grid(data_dict, data.shape, bbox=bbox)

In [ ]:
slc = yt.SlicePlot(ds, "z", ("gas", "density"))
slc.set_cmap(("gas", "density"), "Blues")
slc.annotate_grids(cmap=None)
slc.show()

In [ ]:
sc = yt.create_scene(ds)

# Get a reference to the VolumeSource associated with this scene
# It is the first source associated with the scene, so we can refer to it
# using index 0.
source = sc[0]

# Set the bounds of the transfer function
source.tfh.set_bounds((1e-23, 1e-20))

# set that the transfer function should be evaluated in log space
source.tfh.set_log(True)

# Make underdense regions appear opaque
source.tfh.grey_opacity = False

# Plot the transfer function, along with the CDF of the density field to
# see how the transfer function corresponds to structure in the CDF
source.tfh.plot("transfer_function.png", profile_field=("gas", "density"))

# save the image, flooring especially bright pixels for better contrast
sc.save("rendering.pdf", sigma_clip=6.0)

In [ ]:
!open rendering.pdf